<a href="https://colab.research.google.com/github/zhiyingproject/tongueDiagnosis/blob/main/imgSeg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This notebook is to train a model for locating tongues from people's faces. It is for tongue diagnosis using traditional Chinese Medical Theory (TCM)
!pip uninstall keras-nightly
!pip install h5py==2.10.0
!pip install --upgrade tensorflow==1.15.0
!pip install --upgrade tensorflow-gpu==1.15
# !pip install --upgrade keras==2.2.4
!python -m pip install -U scikit-image==0.16.2
# !pip uninstall keras 
!pip install keras==2.2.4
!pip install wcmatch
# %tensorflow_version 1.x

In [ ]:
# Restarting the kernel to enable tensorflow v1.14
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [ ]:
# Install Mask RCNN
!git clone https://github.com/matterport/Mask_RCNN.git

In [ ]:
cd Mask_RCNN/

/content/Mask_RCNN


In [ ]:
# Setup libraries for Mas RCNN
!python setup.py install
!pip show mask-rcnn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import pathlib
import shutil
import cv2
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import sys
from mrcnn.utils import Dataset, extract_bboxes, compute_ap
from mrcnn.config import Config
from mrcnn.visualize import display_instances
from numpy import zeros, asarray
from tensorflow.keras.utils import plot_model
from mrcnn.model import MaskRCNN, load_image_gt, mold_image
from numpy import zeros, asarray, expand_dims, mean
import math

Using TensorFlow backend.


In [ ]:

from wcmatch.pathlib import Path
ds_path = '/content/drive/MyDrive/backup/tongueDiagnosis/segTraining'
num_files = len(list(Path(ds_path).rglob(['*.jpg', '*.JPG', '*.jpeg'])))
training_threshold = int(num_files * 0.9)
print(training_threshold, num_files - training_threshold)
model_path = '/content/drive/MyDrive/backup/tongueDiagnosis/models'

485 54


In [ ]:
class tongueDataset(Dataset):
    def load_dataset(self, dataset_dir, is_train=True):
        self.add_class("dataset", 1, "tongue")

        images_dir = dataset_dir
        annotations_dir = f'{dataset_dir}/annots'
        image_files = Path(dataset_dir).rglob(['*.jpg','*.JPG','*.jpeg'])
        for ifile, filename in enumerate(list(image_files)):
            image_id = filename.stem
            if is_train and ifile >= training_threshold:
                continue
            if not is_train and ifile < training_threshold:
                continue
            img_path = filename
            ann_path = f'{annotations_dir}/{filename.stem}.xml'
            self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)

    def extract_boxes(self, filename):
        tree = ET.parse(filename)
        root = tree.getroot()
        boxes = list()
        for box in root.findall('.//bndbox'):
            xmin = int(box.find('xmin').text)
            ymin = int(box.find('ymin').text)
            xmax = int(box.find('xmax').text)
            ymax = int(box.find('ymax').text)
            coors = [xmin, ymin, xmax, ymax]
            boxes.append(coors)
        width = int(root.find('.//size/width').text)
        height = int(root.find('.//size/height').text)
        return boxes, width, height

    def load_mask(self, image_id):
        info = self.image_info[image_id]
        path = info['annotation']
        boxes, w, h = self.extract_boxes(path)
        masks = zeros([h, w, len(boxes)], dtype='uint8')

        class_ids = []
        for i in range(len(boxes)):
            box = boxes[i]
            row_s, row_e = box[1], box[3]
            col_s, col_e = box[0], box[2]
            masks[row_s:row_e, col_s:col_e, i] = 1
            class_ids.append(self.class_names.index('tongue'))
        return masks, asarray(class_ids, dtype='int32')

    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']

In [ ]:
class tongueConfig(Config):
    NAME = 'tongue_cfg'
    NUM_CLASSES = 1 + 1
    STEPS_PER_EPOCH = training_threshold
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

In [ ]:
training_ds = tongueDataset()
training_ds.load_dataset(ds_path, is_train=True)
training_ds.prepare()
print('Train: %d' % len(training_ds.image_ids))

test_ds = tongueDataset()
test_ds.load_dataset(ds_path, is_train=False)
test_ds.prepare()
print('Test: %d' % len(test_ds.image_ids))

Train: 485
Test: 54


In [ ]:
config = tongueConfig()
config.display()

model = MaskRCNN(mode='training', model_dir=model_path, config=config)
model.load_weights('/content/drive/MyDrive/backup/tongueDiagnosis/models/mask_rcnn_coco.h5', 
                   by_name=True,
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])
model.train(training_ds, test_ds, learning_rate=config.LEARNING_RATE, epochs=16, layers='heads')


In [ ]:
class predictionConfig(Config):
    NAME = 'tongue_cfg'
    NUM_CLASSES = 1 + 1
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

In [ ]:
def evaluate_model(ds, model, cfg):
    APs = list()
    for i, img_id in enumerate(ds.image_ids):
        print(ds.image_info[i])
        image, image_meta, gt_class_id, gt_bbox, gt_mask = load_image_gt(ds, cfg, img_id, use_mini_mask=False)
        scaled_image = mold_image(image, cfg)
        sample = expand_dims(scaled_image, 0)
        yhat = model.detect(sample, verbose=0)
        r = yhat[0]
        AP, _, _, _ = compute_ap(gt_bbox, gt_class_id, gt_mask, r['rois'], r['class_ids'], r['scores'], r['masks'],0.65)
        print(AP)
        if math.isnan(AP):
            continue
        APs.append(AP)
    mAP = mean(APs)
    return mAP

In [ ]:

training_ds = tongueDataset()
training_ds.load_dataset(ds_path, is_train=True)
training_ds.prepare()
print('Train: %d' % len(training_ds.image_ids))

test_ds = tongueDataset()
test_ds.load_dataset(ds_path, is_train=False)
test_ds.prepare()
print('Test: %d' % len(test_ds.image_ids))


weights_file = '/content/drive/MyDrive/backup/tongueDiagnosis/models/mask_rcnn_tongue_cfg_0016.h5'  # !!!!!need to identify
print(model_path)
cfg_pred = predictionConfig()
model = MaskRCNN(mode='inference', model_dir=model_path, config=cfg_pred)
model.load_weights(weights_file, by_name=True)

In [ ]:
train_mAP = evaluate_model(test_ds, model, cfg_pred)
print("Test mAP: %.3f" % train_mAP)

In [ ]:
def plot_actual_vs_predicted(ds, model, cfg, start, end):
    i = 0
    for i_img in range(start, end):
        image = ds.load_image(i_img)
        n_image = end - start + 1
        print(ds.image_info[i_img]['id'])
        mask, _ = ds.load_mask(i_img)
        scaled_image = mold_image(image, cfg)
        sample = expand_dims(scaled_image, 0)
        yhat = model.detect(sample, verbose=0)[0]
        plt.subplot(n_image, 2, i*2+1)
        plt.imshow(image)
        # plt.title('Actual')
        for j in range(mask.shape[2]):
            plt.imshow(mask[:, :, j], cmap='gray', alpha=0.3)
        plt.subplot(n_image, 2, i*2+2)
        plt.imshow(image)
        # plt.title('Predict')
        ax = plt.gca()
        for box in yhat['rois']:
            y1, x1, y2, x2 = box
            width, height = x2 - x1, y2 - y1
            rect = Rectangle((x1, y1), width, height, fill=False, color='red')
            ax.add_patch(rect)
        i += 1
    plt.show()

In [ ]:
from matplotlib.patches import Rectangle
plot_actual_vs_predicted(test_ds, model, cfg_pred, 5,10)